In [1]:
cd "/home/nicksung/Desktop/nicksung/bwb/data_v4"

/home/nicksung/Desktop/nicksung/bwb/data_v4


In [2]:
import os
import csv
import h5py
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, random_split

#############################
# 1) Dataset & Data Loading #
#############################

class DesignDataset(Dataset):
    """
    (Identical to what you used in training)
    """
    def __init__(self, csv_path, hdf5_path, num_designs=99):
        super().__init__()
        self.num_designs = num_designs

        # 1a) Read CSV
        df = pd.read_csv(csv_path)
        df.rename(columns={"CASE NO": "case_index"}, inplace=True)
        df = df[~df["case_index"].between(390, 399)]  # remove bad cases

        # 1b) Open HDF5
        h5f = h5py.File(hdf5_path, "r")
        points_grp = h5f["points"]
        cp_grp     = h5f["cp"]
        cfx_grp    = h5f["cf_x"]
        cfy_grp    = h5f["cf_y"]
        cfz_grp    = h5f["cf_z"]

        shape_cols  = ['B1', 'B2', 'B3', 'C1', 'C2', 'C3', 'C4', 'S1', 'S2', 'S3']
        flight_cols = ['alt_kft', 'Re_L', 'M_inf', 'alpha_deg', 'beta_deg']

        # Compute normalization for shape + flight
        shape_mean, shape_std = df[shape_cols].mean().values, df[shape_cols].std().values
        flight_mean, flight_std = df[flight_cols].mean().values, df[flight_cols].std().values
        shape_std[shape_std == 0] = 1
        flight_std[flight_std == 0] = 1

        # Find min/max coords and gather all outputs for normalization
        coord_min = np.array([ float("inf"),  float("inf"),  float("inf")])
        coord_max = np.array([-float("inf"), -float("inf"), -float("inf")])
        all_outputs = []
        for case_id in df["case_index"]:
            ds_key = f"case_{case_id:03d}"
            if ds_key in points_grp:
                coords = points_grp[ds_key][()]
                coord_min = np.minimum(coord_min, coords.min(axis=0))
                coord_max = np.maximum(coord_max, coords.max(axis=0))

                cp_data  = cp_grp[ds_key][()]
                cfx_data = cfx_grp[ds_key][()]
                cfy_data = cfy_grp[ds_key][()]
                cfz_data = cfz_grp[ds_key][()]
                all_outputs.append(np.stack([cp_data, cfx_data, cfy_data, cfz_data], axis=-1))

        all_outputs = np.concatenate(all_outputs, axis=0)  # (TotalPoints, 4)
        output_mean = all_outputs.mean(axis=0)
        output_std  = all_outputs.std(axis=0)
        output_std[output_std == 0] = 1

        # Store for use in unnormalization
        self.shape_mean, self.shape_std = shape_mean, shape_std
        self.flight_mean, self.flight_std = flight_mean, flight_std
        self.coord_min, self.coord_max = coord_min, coord_max
        self.output_mean, self.output_std = output_mean, output_std

        # Build the final dataset structure
        tmp_designs = {i: [] for i in range(self.num_designs)}

        for row in df.itertuples(index=False):
            cidx = row.case_index
            design_idx = cidx // 10
            if design_idx >= self.num_designs:
                continue

            ds_key = f"case_{cidx:03d}"
            if ds_key not in points_grp:
                continue

            # flight+shape => normalized
            flight_arr = np.array([row.alt_kft, row.Re_L, row.M_inf, row.alpha_deg, row.beta_deg], dtype=np.float32)
            shape_arr  = np.array([row.B1, row.B2, row.B3, row.C1, row.C2, row.C3, row.C4, row.S1, row.S2, row.S3], dtype=np.float32)
            flight_arr = (flight_arr - flight_mean) / flight_std
            shape_arr  = (shape_arr  - shape_mean)  / shape_std
            full_cond  = np.concatenate([flight_arr, shape_arr]).astype(np.float32)

            # coords => normalized
            pts_coords = points_grp[ds_key][()]
            pts_coords = 2.0 * (pts_coords - coord_min) / (coord_max - coord_min) - 1.0

            # coeffs => normalized
            cp_vals  = (cp_grp[ds_key][()]  - output_mean[0]) / output_std[0]
            cfx_vals = (cfx_grp[ds_key][()] - output_mean[1]) / output_std[1]
            cfy_vals = (cfy_grp[ds_key][()] - output_mean[2]) / output_std[2]
            cfz_vals = (cfz_grp[ds_key][()] - output_mean[3]) / output_std[3]
            coeffs_4 = np.stack([cp_vals, cfx_vals, cfy_vals, cfz_vals], axis=-1)

            tmp_designs[design_idx].append({
                'flight_cond': full_cond,
                'points': pts_coords.astype(np.float32),
                'coeffs': coeffs_4.astype(np.float32)
            })

        # Keep only complete designs (10 flight conditions)
        self.design_info = [tmp_designs[i] for i in range(self.num_designs) if len(tmp_designs[i]) == 10]
        self.num_designs = len(self.design_info)

        h5f.close()

    def __len__(self):
        return self.num_designs

    def __getitem__(self, idx):
        return self.design_info[idx]


################################################
# 2) Train/Val/Test Splitting (Design-level)
################################################
def split_designs(full_dataset, train_ratio=0.8, val_ratio=0.1):
    n_total = len(full_dataset)
    n_train = int(train_ratio * n_total)
    n_val   = int(val_ratio   * n_total)
    n_test  = n_total - n_train - n_val
    train_ds, val_ds, test_ds = random_split(
        full_dataset, [n_train, n_val, n_test],
        generator=torch.Generator().manual_seed(42)
    )
    return train_ds, val_ds, test_ds


######################################
# 3) Define the FiLM-based MLP Model #
######################################
class FiLMModulation(nn.Module):
    def __init__(self, cond_dim, hidden_dim=256, num_layers=4):
        super().__init__()
        self.cond_dim = cond_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        # scale+shift => 2 * hidden_dim * (num_layers - 1)
        self.num_mod_params = 2 * hidden_dim * (num_layers - 1)

        self.fc = nn.Sequential(
            nn.Linear(cond_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, self.num_mod_params)
        )

    def forward(self, cond):
        out = self.fc(cond)
        chunk_size = self.hidden_dim*(self.num_layers-1)
        gamma = out[:, :chunk_size]
        beta  = out[:,  chunk_size:]
        return gamma, beta


class ModulatedMLP(nn.Module):
    def __init__(self, input_dim=3, output_dim=4, hidden_dim=256, num_layers=4):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        # Build layers
        self.layers = nn.ModuleList()
        self.layers.append(nn.Linear(input_dim, hidden_dim))
        for _ in range(num_layers - 2):
            self.layers.append(nn.Linear(hidden_dim, hidden_dim))
        self.layers.append(nn.Linear(hidden_dim, output_dim))

    def forward(self, coords, gamma, beta):
        chunk_size = self.hidden_dim
        h = coords
        for i in range(self.num_layers - 1):
            h = self.layers[i](h)
            h = torch.relu(h)
            g_i = gamma[:, i*chunk_size:(i+1)*chunk_size]
            b_i = beta[:,  i*chunk_size:(i+1)*chunk_size]
            h   = g_i * h + b_i
        out = self.layers[-1](h)  # final
        return out

class AeroFiLMNet(nn.Module):
    def __init__(self, cond_dim=15, coord_dim=3, output_dim=4, hidden_dim=256, num_layers=4):
        super().__init__()
        self.modulation_net = FiLMModulation(cond_dim, hidden_dim, num_layers)
        self.mlp = ModulatedMLP(coord_dim, output_dim, hidden_dim, num_layers)

    def forward(self, coords, cond):
        gamma, beta = self.modulation_net(cond)
        out = self.mlp(coords, gamma, beta)
        return out


############################################################
# 4) Utility: load a trained model checkpoint
############################################################
def load_trained_model(model_path, device='cuda'):
    cond_dim   = 15
    coord_dim  = 3
    output_dim = 4
    hidden_dim = 256
    num_layers = 4
    model = AeroFiLMNet(cond_dim, coord_dim, output_dim, hidden_dim, num_layers)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()
    return model


###########################################################
# 5) Utility: get all points for a given design+case
###########################################################
def get_all_points_for_design_case(dataset, design_idx, case_idx):
    """
    Returns (coords_norm, coeffs_norm, cond_norm),
      coords_norm: (N, 3)
      coeffs_norm: (N, 4)
      cond_norm   : (15,)
    """
    data_dict = dataset.design_info[design_idx][case_idx]
    coords = data_dict['points']      # shape (N,3) normalized
    coeffs = data_dict['coeffs']      # shape (N,4) normalized
    cond   = data_dict['flight_cond'] # shape (15,) normalized
    return coords, coeffs, cond


#############################################################
# 6) Inverse normalization for the aerodynamic coefficients
#############################################################
def unnormalize_coeffs(coeffs_norm, dataset):
    """
    coeffs_norm: shape (N,4), each row = [cp, cfx, cfy, cfz] (normalized)
    returns unnormalized (N,4).
    """
    cp  = coeffs_norm[:, 0] * dataset.output_std[0] + dataset.output_mean[0]
    cfx = coeffs_norm[:, 1] * dataset.output_std[1] + dataset.output_mean[1]
    cfy = coeffs_norm[:, 2] * dataset.output_std[2] + dataset.output_mean[2]
    cfz = coeffs_norm[:, 3] * dataset.output_std[3] + dataset.output_mean[3]
    return np.column_stack([cp, cfx, cfy, cfz])


#########################################################
# 7) Compute the unnormalized MSE across the entire test set
#########################################################
def compute_unnormalized_mse(model, dataset, test_design_indices, device='cuda'):
    """
    Loops over the test design indices & each design's 10 flight conditions,
    gathers all points, predicts & unnormalizes them, compares to GT (also unnormalized),
    and accumulates total squared error. Returns final MSE for each dimension (cp, cf_x, cf_y, cf_z).
    """
    model.eval()
    mse_accum = np.zeros(4, dtype=np.float64)  # sum of squared errors for cp, cfx, cfy, cfz
    count     = 0                              # total number of points

    with torch.no_grad():
        for d_idx in test_design_indices:
            # Each design has 10 flight conditions
            for c_idx in range(10):
                coords_norm, gt_norm, cond_norm = get_all_points_for_design_case(dataset, d_idx, c_idx)

                # Convert to torch
                coords_torch = torch.from_numpy(coords_norm).float().to(device)
                cond_torch   = torch.from_numpy(cond_norm).float().unsqueeze(0).to(device)
                cond_torch   = cond_torch.expand(coords_torch.shape[0], -1)

                preds_norm = model(coords_torch, cond_torch)  # (N,4)
                preds_norm = preds_norm.cpu().numpy()

                # Unnormalize
                preds_unnorm = unnormalize_coeffs(preds_norm, dataset)  # (N,4)
                gt_unnorm    = unnormalize_coeffs(gt_norm, dataset)      # (N,4)

                # Accumulate SSE
                diff = preds_unnorm - gt_unnorm  # (N,4)
                squared_diff = diff**2           # (N,4)
                mse_accum += np.sum(squared_diff, axis=0)  # sum over points
                count += coords_norm.shape[0]

    # Now compute MSE per dimension
    mse_per_dim = mse_accum / count  # (4,) => [MSE_cp, MSE_cfx, MSE_cfy, MSE_cfz]
    return mse_per_dim


################################################
# 8) Main
################################################
def main():
    # File paths
    csv_file   = "/home/nicksung/Desktop/nicksung/bwb/data_v4/parametric/modified_merged_params.csv"
    hdf5_file  = "/home/nicksung/Desktop/nicksung/bwb/data_v4/surface_point/surface_data.hdf5"
    model_file = "/home/nicksung/Desktop/nicksung/bwb/film_model_saved_weights/film_model.pth"   # <-- your saved checkpoint
    device     = "cuda" if torch.cuda.is_available() else "cpu"

    # 1) Build the full dataset
    full_dataset = DesignDataset(csv_file, hdf5_file, num_designs=99)

    # 2) Split into train/val/test
    train_ds, val_ds, test_ds = split_designs(full_dataset, train_ratio=0.8, val_ratio=0.1)
    print("Train designs:", len(train_ds), "Val designs:", len(val_ds), "Test designs:", len(test_ds))

    # 3) We only need the design indices of test_ds
    #    random_split() returns Subset objects that store indices in .indices
    test_indices = test_ds.indices  # This is a list of design indices in the full dataset

    # 4) Load the trained model
    model = load_trained_model(model_file, device=device)

    # 5) Compute unnormalized MSE on all test designs
    mse_cp, mse_cfx, mse_cfy, mse_cfz = compute_unnormalized_mse(model, full_dataset, test_indices, device=device)

    print("Unnormalized MSE for test set:")
    print(f"  cp  : {mse_cp:.6e}")
    print(f"  cf_x: {mse_cfx:.6e}")
    print(f"  cf_y: {mse_cfy:.6e}")
    print(f"  cf_z: {mse_cfz:.6e}")

    # If desired, you can also compute an overall average MSE across the 4 dims:
    avg_mse = (mse_cp + mse_cfx + mse_cfy + mse_cfz) / 4.0
    print(f"Average unnormalized MSE across (cp, cf_x, cf_y, cf_z): {avg_mse:.6e}")




In [3]:
main()

Train designs: 78 Val designs: 9 Test designs: 11
Unnormalized MSE for test set:
  cp  : 2.938683e-02
  cf_x: 1.023803e-06
  cf_y: 5.246763e-07
  cf_z: 9.456416e-07
Average unnormalized MSE across (cp, cf_x, cf_y, cf_z): 7.347330e-03


In [4]:
import numpy as np

def compute_unnormalized_stats(dataset):
    """
    Loops over all designs & flight conditions in `dataset.design_info`.
    For each (design, case), retrieves the normalized coeffs -> unnormalizes
    -> stores them in big arrays.

    Then prints out mean, std, min, max for each of (cp, cf_x, cf_y, cf_z).
    """
    all_cp   = []
    all_cfx  = []
    all_cfy  = []
    all_cfz  = []

    for d_idx in range(dataset.num_designs):
        # Each design_info[d_idx] is a list of 10 flight conditions
        design_cases = dataset.design_info[d_idx]
        for case_data in design_cases:
            # case_data is a dict with 'points', 'coeffs' (normalized), 'flight_cond'
            coeffs_norm = case_data['coeffs']  # shape (N,4)

            # Unnormalize them:
            #   cp_unnorm = cp_norm * std_cp + mean_cp, etc.
            cp_unnorm  = coeffs_norm[:, 0] * dataset.output_std[0] + dataset.output_mean[0]
            cfx_unnorm = coeffs_norm[:, 1] * dataset.output_std[1] + dataset.output_mean[1]
            cfy_unnorm = coeffs_norm[:, 2] * dataset.output_std[2] + dataset.output_mean[2]
            cfz_unnorm = coeffs_norm[:, 3] * dataset.output_std[3] + dataset.output_mean[3]

            all_cp.append(cp_unnorm)
            all_cfx.append(cfx_unnorm)
            all_cfy.append(cfy_unnorm)
            all_cfz.append(cfz_unnorm)

    # Concatenate all points from all designs/cases
    all_cp   = np.concatenate(all_cp)
    all_cfx  = np.concatenate(all_cfx)
    all_cfy  = np.concatenate(all_cfy)
    all_cfz  = np.concatenate(all_cfz)

    # Compute basic stats
    print("==== Unnormalized Stats for Entire Dataset ====")
    print(f"CP   : mean={all_cp.mean():.5f},  std={all_cp.std():.5f},  min={all_cp.min():.5f},  max={all_cp.max():.5f}")
    print(f"CF_x : mean={all_cfx.mean():.5f}, std={all_cfx.std():.5f}, min={all_cfx.min():.5f}, max={all_cfx.max():.5f}")
    print(f"CF_y : mean={all_cfy.mean():.5f}, std={all_cfy.std():.5f}, min={all_cfy.min():.5f}, max={all_cfy.max():.5f}")
    print(f"CF_z : mean={all_cfz.mean():.5f}, std={all_cfz.std():.5f}, min={all_cfz.min():.5f}, max={all_cfz.max():.5f}")


In [6]:
csv_file   = "/home/nicksung/Desktop/nicksung/bwb/data_v4/parametric/modified_merged_params.csv"
hdf5_file  = "/home/nicksung/Desktop/nicksung/bwb/data_v4/surface_point/surface_data.hdf5"

# 1) Build the dataset (this computes all your normalization info)
full_dataset = DesignDataset(csv_file, hdf5_file, num_designs=99)

# 2) Print the stored means & stds directly:
print("Raw means from the dataset:", full_dataset.output_mean)  # shape (4,)
print("Raw stds from the dataset :", full_dataset.output_std)   # shape (4,)

# 3) Use our helper function to see the distribution (min, max, etc.)
compute_unnormalized_stats(full_dataset)

Raw means from the dataset: [-1.14445232e-01  3.33085008e-03 -1.96375195e-06  5.94268788e-04]
Raw stds from the dataset : [0.45769806 0.00254975 0.00147762 0.00324235]
==== Unnormalized Stats for Entire Dataset ====
CP   : mean=-0.11456,  std=0.45777,  min=-7.95721,  max=1.68977
CF_x : mean=0.00333, std=0.00255, min=-0.01830, max=0.03099
CF_y : mean=-0.00000, std=0.00148, min=-0.03001, max=0.03182
CF_z : mean=0.00059, std=0.00324, min=-0.01891, max=0.09298
